In [1]:
from azureml.core import Workspace, Experiment

#setting workspace and experiment, can be seen in Azure WS
ws = Workspace.get(name="quick-starts-ws-126643")
exp = Experiment(workspace=ws, name="udacity-project-Saad")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-126643
Azure region: southcentralus
Subscription id: 888519c8-2387-461a-aff3-b31b86e2438e
Resource group: aml-quickstarts-126643


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# DONE: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
## as per labs and Azure ML documentations 
compute_name = "amlcompute1"
try:
    compute_cluster = ComputeTarget(ws, compute_name)
    print(f"{compute_name} exists already")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, compute_name, compute_config)
compute_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [11]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
import os

# Specify parameter sampler
#can try this with Grid Sampling and Bayesian Sampling as well
ps = RandomParameterSampling({"--C": choice(1,3,5,7), "--max_iter": choice(75,100,125,150,175)})

# Specify a Policy
#slack_factor used to calculate the allowed distance from the best performing run
#delay_evaluation for delay in evaluation_interval
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, delay_evaluation=3)

#creating a training folder if not already exist
if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py 
#using default params of train.py
est = SKLearn(source_directory='./', compute_target=compute_cluster, entry_script='train.py', script_params={
    '--C':1,'--max_iter':100})

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
#max_total_runs to be updated later if project takes more time than allocated
hyperdrive_config = HyperDriveConfig(estimator=est, policy=policy, primary_metric_name="Accuracy",
                                     hyperparameter_sampling=ps, max_total_runs=20,
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE)

In [12]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.
WARNING - If 'arguments' has been provided here and arguments have been specified in 'run_config', 'arguments' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [13]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
#getting best run and metric via hyper drive lab instructions
run_top = hyperdrive_run.get_best_run_by_primary_metric()
run_top_metric = run_top.get_metrics()

#printing top run id and mertic
print(run_top.id)
print(run_top_metric['Accuracy'])

#saving the model from top run
best_model = run_top.register_model(model_name = 'best_model_acc', model_path = './')

HD_0e1515c1-b529-4f1f-b5de-9893dd1c50b1_17
0.9134498664724447


## Part 2 AutoML Run

In [14]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = TabularDatasetFactory.from_delimited_files(data)

In [17]:
from train import clean_data

# Use the clean_data function to clean your data.
#train.py script will be called and same function from that will clean the data
x, y = clean_data(ds)

In [18]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    compute_target = compute_cluster,
    task='classification',
    primary_metric='accuracy',
    training_data= ds,
    label_column_name='y',
    n_cross_validations=4)
#changed n_cross_validation, given value was 7

In [20]:
# Submit your automl run

#for timeout issue after hyperdrive run, if required re-importing
#from azureml.core.experiment import Experiment
#from azureml.widgets import RunDetails

### YOUR CODE HERE ###
exp2_automl = Experiment(ws, "automl_Saad")
exp2_run = exp2_automl.submit(config = automl_config, show_output = True)
RunDetails(exp2_run).show()

Running on remote.
Running on remote compute: amlcompute1
Parent Run ID: AutoML_08013071-4802-4c0d-b841-432ce016f5d5

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has b

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [21]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
top_run, top_model = exp2_run.get_output()

print(top_run)

top_run.register_model(model_name = 'best_automlmodel_exp2.pkl', model_path = './outputs/')

Run(Experiment: automl_Saad,
Id: AutoML_08013071-4802-4c0d-b841-432ce016f5d5_25,
Type: azureml.scriptrun,
Status: Completed)


Model(workspace=Workspace.create(name='quick-starts-ws-126643', subscription_id='888519c8-2387-461a-aff3-b31b86e2438e', resource_group='aml-quickstarts-126643'), name=best_automlmodel_exp2.pkl, id=best_automlmodel_exp2.pkl:1, version=1, tags={}, properties={})

In [22]:
#deleting the compute cluster
compute_cluster.delete()